In [23]:
from selenium import webdriver
from selenium.webdriver import ChromeOptions
import time
from bs4 import BeautifulSoup

In [24]:
options = ChromeOptions()
options.add_argument("headless")
options.add_argument("disable-gpu")

In [29]:
driver = webdriver.Chrome(options=options)
driver.get('https://www.ftserussell.com/analytics/factsheets/home/search')
archive = driver.find_element_by_xpath('//a[@title="FTSE US Broad Investment-Grade Bond Index (USBIG®)"]/following-sibling::a')
driver.execute_script("arguments[0].click();", archive)
time.sleep(2)
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.close()
div = soup.find("h4", string="FTSE Global All Cap ex US Index").parent
for li in div.find('ul').findAll('li'):
    links.append(li.find('a',href=True)['href'])
links.append(BeautifulSoup(requests.get('https://www.ftserussell.com/analytics/factsheets/home/search').text,features="html.parser").find('a',{'title':'FTSE Global All Cap ex US Index'})['href'])

AttributeError: 'NoneType' object has no attribute 'parent'

In [31]:
soup.find("h4", string="FTSE Global All Cap ex US Index")

In [ ]:
for link in links:
    pdf = requests.get(link)
    open_pdf = io.BytesIO(pdf.content)
    read_pdf = PDF(open_pdf)
    dates.append(re.sub('Dataasat:','',re.findall('Dataasat:\d+\S+\d+', re.sub(' ','',read_pdf[0]), re.IGNORECASE)[0]))
    marketCaps.append(float(re.sub('Totals\s+\d+\s+','',re.findall('Totals\s+\d+\s+\d+,\d+,\d+', read_pdf[1], re.IGNORECASE)[0]).replace(',',''))/1000)
vxusDF = pd.DataFrame({'date':pd.to_datetime(dates, infer_datetime_format=True),'Market Cap':marketCaps}).sort_values(by='date').reset_index(drop=True)
vxusDF.to_csv('Data/vxus_MC_data.csv',index=False)
print("VXUS")
print(vxusDF)